In [1]:
# ===================== IMPORTS/LIBRARIES =====================
from luxpy import (cat, colortf, _CIEOBS, _CIE_ILLUMINANTS, _CRI_RFL, _CIE_D65,_CIE_E,
                   spd_to_xyz, plot_color_data, math, cie_interp, getwlr, xyz_to_srgb)
from luxpy.utils import np, plt, sp, _PKG_PATH, _SEP, _EPS 
import warnings
from imageio import imsave

import tensorflow as tf
from mtcnn import MTCNN
from pathlib import Path
import pandas as pd
import glob
import cv2
import csv
import os
import ast
import pydot
import pydotplus
import graphviz
from datetime import datetime
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback
from keras.utils.vis_utils import plot_model

In [12]:
# ===================== CSV HISTORY FUNCTIONS =====================

def historyToCsv(history):
    # convert the history.history dict to a pandas DataFrame:     
    hist_df = pd.DataFrame(history.history) 
    
    current_datetime = datetime.now()

    # Convert the datetime object to a string
    filename_friendly_datetime_string = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")
    
    # save to csv: 
    hist_csv_file = 'history' + filename_friendly_datetime_string + '.csv'
    with open(hist_csv_file, mode='w') as f:
        hist_df.to_csv(f)

def csvToHistory(csv_filename):
    # Read the CSV file into a pandas DataFrame
    hist_df = pd.read_csv(csv_filename, index_col=0)

    # Convert the DataFrame to a dictionary
    history_dict = hist_df.to_dict(orient='list')

    return history_dict

In [33]:
# add to imports
from tensorflow.keras.optimizers import Adam

# ===================== MULTITASK MODEL SETUP =====================
base_model = tf.keras.applications.VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)

flattened_features = tf.keras.layers.Flatten(name='flattened_features')(base_model.output)

embedding_layer = tf.keras.layers.Dense(512, activation='linear', name='embedding')(flattened_features)

additional_dense_layer1 = tf.keras.layers.Dense(64, activation='relu')(embedding_layer)
additional_Dropout_layer1 = tf.keras.layers.Dropout(0.2)(additional_dense_layer1)
additional_dense_layer2 = tf.keras.layers.Dense(64, activation='relu')(additional_Dropout_layer1)
additional_Dropout_layer2 = tf.keras.layers.Dropout(0.2)(additional_dense_layer2)

additional_dense_layer3 = tf.keras.layers.Dense(64, activation='relu')(embedding_layer)
additional_Dropout_layer3 = tf.keras.layers.Dropout(0.2)(additional_dense_layer3)

# landmarks = tf.keras.layers.Dense(10, activation='linear', name='landmark_output')(additional_dense_layer14)
landmarks = tf.keras.layers.Dense(10, activation='linear', name='landmark_output')(additional_dense_layer2)
illum = tf.keras.layers.Dense(1, activation='linear', name='previous_illuminance_output')(additional_dense_layer4)

# Reshape layer to the desired shape
reshaped_features = tf.keras.layers.Reshape((8, 8, 8))(embedding_layer)

# Conv2DTranspose layers instead of UpSampling2D
conv_transpose1 = tf.keras.layers.Conv2DTranspose(8, kernel_size=(3, 3), activation='relu', padding='same', strides=(7, 7))(reshaped_features)
conv_transpose2 = tf.keras.layers.Conv2DTranspose(16, kernel_size=(3, 3), activation='relu', padding='same', strides=(2, 2))(conv_transpose1)
conv_transpose3 = tf.keras.layers.Conv2DTranspose(32, kernel_size=(3, 3), activation='relu', padding='same', strides=(2, 2))(conv_transpose2)

retIllum = tf.keras.layers.Conv2D(3, kernel_size=(3, 3), activation='relu', padding='same', name='image_retinex_output')(conv_transpose3 )

task_outputs = None

task_outputs = [landmarks, illum, retIllum]

multi_task_model = tf.keras.Model(inputs=base_model.input, outputs=task_outputs)

# Compile the model with specific loss functions and metrics for each task
multi_task_model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss={
        'landmark_output': 'mean_squared_error',
        'previous_illuminance_output': 'mean_squared_error',
        'image_retinex_output': 'mean_squared_error'
    },
    metrics={
        'landmark_output': ['mse', "mae"],
        'previous_illuminance_output': ['mse', "mae"],
        'image_retinex_output': ['mse', "mae"]
    }
)

multi_task_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 224, 224, 64  1792        ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 224, 224, 64  36928       ['block1_conv1[0][0]']           
                                )                                                           

In [33]:
base_model = tf.keras.applications.VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)

task_outputs = [landmarks, illum, retIllum]

multi_task_model = tf.keras.Model(inputs=base_model.input, outputs=task_outputs)

multi_task_model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss={
        'landmark_output': 'mean_squared_error',
        'previous_illuminance_output': 'mean_squared_error',
        'image_retinex_output': 'mean_squared_error'
    },
    metrics={
        'landmark_output': ['mse', "mae"],
        'previous_illuminance_output': ['mse', "mae"],
        'image_retinex_output': ['mse', "mae"]
    }
)

multi_task_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 224, 224, 64  1792        ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 224, 224, 64  36928       ['block1_conv1[0][0]']           
                                )                                                           

In [16]:
pip install pydot

Note: you may need to restart the kernel to use updated packages.


In [17]:
import pydot
import graphviz
import matplotlib.pyplot as plt
tf.keras.utils.plot_model(multi_task_model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [5]:
# # ===================== MULTI-TASK DATA GEN =====================

# class CustomDataGen(tf.keras.utils.Sequence):
    
#     def __init__(self, df, X_col, y_col,
#                  batch_size,
#                  input_size=(224, 224, 3),
#                  shuffle=True,
#                  random_seed=None):  # Add a new parameter for random seed
        
#         self.df = df.copy()
#         self.X_col = X_col
#         self.y_col = y_col
#         self.batch_size = batch_size
#         self.input_size = input_size
#         self.shuffle = shuffle
#         self.random_seed = random_seed  # Store the random seed
        
#         self.n = len(self.df)
#         self.n_coords = 2  # Assuming landmark coordinates are 2-dimensional
#         self.n_illuminance = 1  # Assuming a single illuminance value
    
#     def on_epoch_end(self):
#         if self.shuffle:
#             self.df = self.df.sample(frac=1, random_state=self.random_seed).reset_index(drop=True)  # Use the random seed
    
#     def __get_input(self, path, target_size):
    
#         image = tf.keras.preprocessing.image.load_img(path)
#         image_arr = tf.keras.preprocessing.image.img_to_array(image)

#         image_arr = tf.image.resize(image_arr, (target_size[0], target_size[1])).numpy()

#         return image_arr / 255.
    
#     def __get_output(self, label, output_type):
#         # Assuming output_type is 'coordinates', 'illuminance', or 'adjusted_image_path'
#         if output_type == 'coordinates':
#             # Assuming label is a string containing a dictionary-like structure
#             # Safely evaluate the string as a literal dictionary using ast.literal_eval
#             coordinates_dict = ast.literal_eval(label)
            
#             # Extract x and y coordinates for each landmark
#             landmarks = ['left_eye', 'right_eye', 'nose', 'mouth_left', 'mouth_right']
#             coordinates_list = [coordinates_dict[landmark] for landmark in landmarks]
            
#             # Flatten the list and convert to numpy array
#             coordinates_array = np.array([coord for landmark_coords in coordinates_list for coord in landmark_coords])
#             print(f'original coords: {coordinates_array}')
            
# #             print("Shape of landmarks_array:", coordinates_array.shape)
            
#             # If there are exactly 10 values, return the array, otherwise raise an error
#             if len(coordinates_array) == 10:
#                 return coordinates_array
#             else:
#                 raise ValueError("Expected 10 coordinates, but found {}".format(len(coordinates_array)))
#         elif output_type == 'illuminance':
#             # Convert the illuminance value to a float
#             return float(label)
#         elif output_type == 'adjusted_image_path':
#             # Assuming label is the path to the adjusted image
#             return self.__get_input(label, self.input_size)
    
#     def __get_data(self, batches):
#         # Generates data containing batch_size samples

#         path_batch = batches[self.X_col['path']]
        
#         coords_batch = batches[self.y_col['coordinates']]
#         illuminance_batch = batches[self.y_col['illuminance']]
#         adjusted_image_path_batch = batches[self.y_col['adjusted_image_path']]

#         X_batch = np.asarray([self.__get_input(x, self.input_size) for x in path_batch])

#         y0_batch = np.asarray([self.__get_output(y, 'coordinates') for y in coords_batch])
#         y1_batch = np.asarray([self.__get_output(y, 'illuminance') for y in illuminance_batch])
#         y2_batch = np.asarray([self.__get_output(y, 'adjusted_image_path') for y in adjusted_image_path_batch])
        
#         return X_batch, [y0_batch, y1_batch, y2_batch]
    
#     def __getitem__(self, index):
#         batches = self.df[index * self.batch_size:(index + 1) * self.batch_size]
#         X, y = self.__get_data(batches)
        
#         return X, y
    
#     def __len__(self):
#         return self.n // self.batch_size

In [36]:
# ===================== MULTI-TASK DATA GEN =====================

class CustomDataGen(tf.keras.utils.Sequence):
    
    def __init__(self, df, X_col, y_col,
                 batch_size,
                 input_size=(224, 224, 3),
                 shuffle=True,
                 random_seed=None):  # Add a new parameter for random seed
        
        self.df = df.copy()
        self.X_col = X_col
        self.y_col = y_col
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        self.random_seed = random_seed  # Store the random seed
        
        self.n = len(self.df)
        self.n_coords = 2  # Assuming landmark coordinates are 2-dimensional
        self.n_illuminance = 1  # Assuming a single illuminance value
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1, random_state=self.random_seed).reset_index(drop=True)  # Use the random seed
    
    def __get_input(self, path, target_size):
    
        image = tf.keras.preprocessing.image.load_img(path)
        image_arr = tf.keras.preprocessing.image.img_to_array(image)

        image_arr = tf.image.resize(image_arr, (target_size[0], target_size[1])).numpy()

        return image_arr / 255.
    
    def __get_output(self, label, output_type):
        # Assuming output_type is 'coordinates', 'illuminance', or 'adjusted_image_path'
        if output_type == 'coordinates':
            # Assuming label is a string containing a dictionary-like structure
            # Safely evaluate the string as a literal dictionary using ast.literal_eval
            coordinates_dict = ast.literal_eval(label)
            
            # Extract x and y coordinates for each landmark
            landmarks = ['left_eye', 'right_eye', 'nose', 'mouth_left', 'mouth_right']
            coordinates_list = [coordinates_dict[landmark] for landmark in landmarks]
            
            # Flatten the list and convert to numpy array
            coordinates_array = np.array([coord for landmark_coords in coordinates_list for coord in landmark_coords])
            
#             coordinates_array = normalize_coordinates(coordinates_array, 640, 480)

#             print("Shape of landmarks_array:", coordinates_array.shape)
            
            # If there are exactly 10 values, return the array, otherwise raise an error
            if len(coordinates_array) == 10:
                return coordinates_array
            else:
                raise ValueError("Expected 10 coordinates, but found {}".format(len(coordinates_array)))
        elif output_type == 'illuminance':
            # Convert the illuminance value to a float
            return float(label)
        elif output_type == 'adjusted_image_path':
            # Assuming label is the path to the adjusted image
            return self.__get_input(label, self.input_size)
    
    def __get_data(self, batches):
        # Generates data containing batch_size samples

        path_batch = batches[self.X_col['path']]
        
        coords_batch = batches[self.y_col['coordinates']]
        illuminance_batch = batches[self.y_col['illuminance']]
        adjusted_image_path_batch = batches[self.y_col['adjusted_image_path']]

        X_batch = np.asarray([self.__get_input(x, self.input_size) for x in path_batch])

        y0_batch = np.asarray([self.__get_output(y, 'coordinates') for y in coords_batch])
        y1_batch = np.asarray([self.__get_output(y, 'illuminance') for y in illuminance_batch])
        y2_batch = np.asarray([self.__get_output(y, 'adjusted_image_path') for y in adjusted_image_path_batch])

        return X_batch, [y0_batch, y1_batch, y2_batch]
    
    def __getitem__(self, index):
        batches = self.df[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__get_data(batches)
        
        return X, y
    
    def __len__(self):
        return self.n // self.batch_size

# def normalize_coordinates(coordinates, original_width, original_height):
#     normalized_coordinates = np.zeros_like(coordinates, dtype=np.float32)
    
#     for i in range(0, len(coordinates), 2):
#         x = coordinates[i]
#         y = coordinates[i + 1]
        
#         normalized_x = (x / original_width)
#         normalized_y = (y / original_height)
        
#         normalized_coordinates[i] = normalized_x
#         normalized_coordinates[i + 1] = normalized_y
    
#     return normalized_coordinates

In [37]:
# ===================== DATA GEN SETUP =====================

train_df = pd.read_csv("./data/training.csv") # path to train_data csv
train_df["Filename"] = "./data/Training/" + train_df["Filename"]
train_df["Retinex"] = "./data/RetTraining/retinex_results_train/" + train_df["Retinex"]

# Define column indices or names for X and y
X_col = {'path': 'Filename'}
y_col = {'illuminance': 'Illuminance', 'coordinates': 'Landmarks', 'adjusted_image_path': 'Retinex'}

# Create an instance of CustomDataGen
train_gen = CustomDataGen(train_df, X_col, y_col, batch_size=32, input_size=(224, 224, 3), random_seed=438)

eval_df = pd.read_csv("./data/evaluation.csv") # path to eval_data csv
eval_df["Filename"] = "./data/Evaluation/" + eval_df["Filename"]
eval_df["Retinex"] = "./data/RetEvaluation/retinex_results_eval/" + eval_df["Retinex"]

# Define column indices or names for X and y
eval_X_col = {'path': 'Filename'}
eval_y_col = {'illuminance': 'Illuminance', 'coordinates': 'Landmarks', 'adjusted_image_path': 'Retinex'}

val_gen = CustomDataGen(eval_df, eval_X_col, eval_y_col, batch_size=32, input_size=(224, 224, 3), random_seed=438)
# print(eval_df)
# print(train_df)

In [38]:
print(train_df)

                                                Filename  Illuminance  \
0      ./data/Training/glasses_001_nonsleepyCombinati...     0.295678   
1      ./data/Training/glasses_001_nonsleepyCombinati...     0.296213   
2      ./data/Training/glasses_001_nonsleepyCombinati...     0.296964   
3      ./data/Training/glasses_001_nonsleepyCombinati...     0.296636   
4      ./data/Training/glasses_001_nonsleepyCombinati...     0.297338   
...                                                  ...          ...   
30098   ./data/Training/sunglasses_036_yawning_900_0.jpg     0.301855   
30099    ./data/Training/sunglasses_036_yawning_90_0.jpg     0.303128   
30100   ./data/Training/sunglasses_036_yawning_930_0.jpg     0.301555   
30101   ./data/Training/sunglasses_036_yawning_960_0.jpg     0.299896   
30102   ./data/Training/sunglasses_036_yawning_990_1.jpg     0.301715   

                                               Landmarks  \
0      {'left_eye': (214, 232), 'right_eye': (306, 22...   
1  

In [39]:
print(train_gen.__getitem__(0))

(array([[[[1.0264e-01, 1.0264e-01, 1.0264e-01],
         [1.0420e-01, 1.0420e-01, 1.0420e-01],
         [9.7759e-02, 9.7759e-02, 9.7759e-02],
         ...,
         [6.0392e-01, 6.0392e-01, 6.0392e-01],
         [6.0308e-01, 6.0308e-01, 6.0308e-01],
         [6.0000e-01, 6.0000e-01, 6.0000e-01]],

        [[1.0084e-01, 1.0084e-01, 1.0084e-01],
         [1.0364e-01, 1.0364e-01, 1.0364e-01],
         [1.0292e-01, 1.0292e-01, 1.0292e-01],
         ...,
         [6.0392e-01, 6.0392e-01, 6.0392e-01],
         [6.0280e-01, 6.0280e-01, 6.0280e-01],
         [6.0000e-01, 6.0000e-01, 6.0000e-01]],

        [[1.3645e-01, 1.3645e-01, 1.3645e-01],
         [1.4370e-01, 1.4370e-01, 1.4370e-01],
         [1.5554e-01, 1.5554e-01, 1.5554e-01],
         ...,
         [6.0392e-01, 6.0392e-01, 6.0392e-01],
         [6.0392e-01, 6.0392e-01, 6.0392e-01],
         [6.0392e-01, 6.0392e-01, 6.0392e-01]],

        ...,

        [[2.3529e-02, 2.3529e-02, 2.3529e-02],
         [2.3529e-02, 2.3529e-02, 2.3529e-02

In [35]:
array([[214, 232, 306, 222, 239, 283, 227, 342, 298, 332],
       [210, 242, 300, 229, 235, 293, 226, 353, 296, 343],
       [219, 237, 313, 232, 244, 292, 227, 348, 300, 344],
       [219, 236, 314, 232, 245, 292, 228, 347, 301, 344],
       [220, 236, 314, 231, 245, 292, 229, 348, 301, 344],
       [220, 234, 315, 231, 245, 293, 229, 347, 301, 344],
       [222, 235, 317, 231, 247, 292, 230, 347, 303, 344],
       [221, 234, 316, 231, 246, 293, 229, 348, 302, 345],
       [208, 235, 299, 222, 234, 285, 225, 344, 296, 333],
       [216, 241, 308, 233, 242, 293, 228, 351, 300, 345],
       [217, 241, 309, 233, 242, 293, 229, 351, 301, 344],
       [218, 239, 309, 230, 242, 291, 232, 348, 303, 341],
       [219, 239, 313, 231, 245, 293, 231, 351, 303, 345],

SyntaxError: unexpected EOF while parsing (1208597786.py, line 17)

In [31]:
class DynamicLearningRateScheduler(Callback):
    def __init__(self, monitor_metric='val_loss', patience=3, factor=0.5, min_lr=1e-6):
        super(DynamicLearningRateScheduler, self).__init__()
        self.monitor_metric = monitor_metric
        self.patience = patience
        self.factor = factor
        self.min_lr = min_lr
        self.wait = 0
        self.best_metric = float('inf')

    def on_epoch_end(self, epoch, logs=None):
        current_metric = logs.get(self.monitor_metric)

        if current_metric is None:
            raise ValueError(f"Metric {self.monitor_metric} not found in training logs.")

        if current_metric < self.best_metric:
            self.best_metric = current_metric
            self.wait = 0
        else:
            self.wait += 1
            if self.wait >= self.patience:
                new_lr = max(self.model.optimizer.lr * self.factor, self.min_lr)
                self.model.optimizer.lr = new_lr
                print(f"\nLearning rate reduced to {new_lr}.")
                self.wait = 0

dynamicLearningCallback = DynamicLearningRateScheduler(monitor_metric='val_loss', patience=3, factor=0.5, min_lr=1e-6)

class SaveModelEveryNEpochs(Callback):
    def __init__(self, save_path, save_freq=5):
        super(SaveModelEveryNEpochs, self).__init__()
        self.save_path = save_path
        self.save_freq = save_freq

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.save_freq == 0:
            model_filename = f"{self.save_path}_epoch_{epoch + 1}.keras"
            self.model.save(model_filename)
            print(f"\nModel saved to {model_filename}.")

callback_save_path = "./Callback_Models"  # Change this to your desired save path
saveModelCallback = SaveModelEveryNEpochs(callback_save_path, save_freq=5)

In [ ]:
history = multi_task_model.fit(train_gen, 
                               epochs=20, 
                               validation_data=val_gen, 
                               callbacks=[dynamicLearningCallback])

Epoch 1/5
940/940 [==============================] - 789s 838ms/step - loss: 0.1395 - landmark_output_loss: 0.0225 - previous_illuminance_output_loss: 0.0054 - image_retinex_output_loss: 0.1117 - landmark_output_mse: 0.0225 - landmark_output_mae: 0.0984 - previous_illuminance_output_mse: 0.0054 - previous_illuminance_output_mae: 0.0484 - image_retinex_output_mse: 0.1117 - image_retinex_output_mae: 0.2603 - val_loss: 0.0772 - val_landmark_output_loss: 0.0121 - val_previous_illuminance_output_loss: 0.0013 - val_image_retinex_output_loss: 0.0638 - val_landmark_output_mse: 0.0121 - val_landmark_output_mae: 0.0679 - val_previous_illuminance_output_mse: 0.0013 - val_previous_illuminance_output_mae: 0.0298 - val_image_retinex_output_mse: 0.0638 - val_image_retinex_output_mae: 0.2056
Epoch 2/5
940/940 [==============================] - 780s 830ms/step - loss: 0.0607 - landmark_output_loss: 0.0065 - previous_illuminance_output_loss: 2.7420e-04 - image_retinex_output_loss: 0.0539 - landmark_outp

In [9]:
# ===================== SAVE MODEL WEIGHTS =====================

multi_task_model.save('MultiTaskModel.keras')
history.history

{'loss': [6084.75,
  1293.852294921875,
  1011.7276611328125,
  926.9559326171875,
  850.9852905273438,
  820.2369384765625,
  749.4328002929688,
  711.6307373046875,
  698.1751098632812,
  666.7297973632812,
  507.69110107421875,
  465.22216796875,
  446.80010986328125,
  405.0002746582031,
  406.2571105957031,
  378.9593811035156,
  363.1372985839844,
  280.4102478027344,
  264.2939147949219,
  235.73516845703125,
  184.12384033203125,
  180.36651611328125,
  161.84970092773438,
  139.6802215576172,
  131.44332885742188,
  126.03266906738281,
  110.88644409179688,
  104.33997344970703,
  101.34040832519531,
  96.81082153320312,
  90.68335723876953,
  91.76692199707031,
  88.14944458007812,
  86.65374755859375,
  85.7391586303711,
  84.73784637451172,
  83.7243423461914,
  82.69212341308594,
  81.7379379272461,
  81.11091613769531],
 'landmark_output_loss': [5964.66455078125,
  1293.1578369140625,
  1011.295654296875,
  926.4330444335938,
  850.572509765625,
  819.8648071289062,
  749

In [ ]:
# # ===================== PLOT MODEL =====================
# plt.figure(figsize=(12, 6))
# plt.subplot(1, 2, 1)
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend(['Train', 'Validation'], loc='upper right')

# # Plot training & validation loss values
# plt.figure(figsize=(12, 6))
# plt.subplot(1, 2, 1)
# plt.plot(history.history['landmark_output_loss'])
# plt.plot(history.history['val_landmark_output_loss'])
# plt.title('Landmarks Task Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend(['Train', 'Validation'], loc='upper right')

# # Plot training & validation loss values
# plt.figure(figsize=(12, 6))
# plt.subplot(1, 2, 1)
# plt.plot(history.history['landmark_output_mse'])
# plt.plot(history.history['val_landmark_output_mse'])
# plt.title('Landmarks Task MSE')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend(['Train', 'Validation'], loc='upper right')

# # Plot training & validation loss values
# plt.figure(figsize=(12, 6))
# plt.subplot(1, 2, 1)
# plt.plot(history.history['landmark_output_mae'])
# plt.plot(history.history['val_landmark_output_mae'])
# plt.title('Landmarks Task MAE')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend(['Train', 'Validation'], loc='upper right')

# # Plot training & validation loss values
# plt.figure(figsize=(12, 6))
# plt.subplot(1, 2, 1)
# plt.plot(history.history['previous_illuminance_output_loss'])
# plt.plot(history.history['val_previous_illuminance_output_loss'])
# plt.title('Illuminance Task Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend(['Train', 'Validation'], loc='upper right')

# # Plot training & validation loss values
# plt.figure(figsize=(12, 6))
# plt.subplot(1, 2, 1)
# plt.plot(history.history['previous_illuminance_output_mse'])
# plt.plot(history.history['val_previous_illuminance_output_mse'])
# plt.title('Illuminance Task MSE')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend(['Train', 'Validation'], loc='upper right')

# # Plot training & validation loss values
# plt.figure(figsize=(12, 6))
# plt.subplot(1, 2, 1)
# plt.plot(history.history['previous_illuminance_output_mae'])
# plt.plot(history.history['val_previous_illuminance_output_mae'])
# plt.title('Illuminance Task MAE')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend(['Train', 'Validation'], loc='upper right')

# # Plot training & validation loss values
# plt.figure(figsize=(12, 6))
# plt.subplot(1, 2, 1)
# plt.plot(history.history['image_retinex_output_loss'])
# plt.plot(history.history['val_image_retinex_output_loss'])
# plt.title('Retinex Task Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend(['Train', 'Validation'], loc='upper right')

# # Plot training & validation loss values
# plt.figure(figsize=(12, 6))
# plt.subplot(1, 2, 1)
# plt.plot(history.history['image_retinex_output_mse'])
# plt.plot(history.history['val_image_retinex_output_mse'])
# plt.title('Landmarks Task MSE')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend(['Train', 'Validation'], loc='upper right')

# # Plot training & validation loss values
# plt.figure(figsize=(12, 6))
# plt.subplot(1, 2, 1)
# plt.plot(history.history['image_retinex_output_mae'])
# plt.plot(history.history['val_image_retinex_output_mae'])
# plt.title('Landmarks Task MAE')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend(['Train', 'Validation'], loc='upper right')

# plt.tight_layout()
# plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')

# Plot training & validation loss values
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['landmark_output_loss'])
plt.plot(history.history['val_landmark_output_loss'])
plt.title('Landmarks Task Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')

# Plot training & validation loss values
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['landmark_output_mse'])
plt.plot(history.history['val_landmark_output_mse'])
plt.title('Landmarks Task MSE')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.legend(['Train', 'Validation'], loc='upper right')

# Plot training & validation loss values
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['landmark_output_mae'])
plt.plot(history.history['val_landmark_output_mae'])
plt.title('Landmarks Task MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend(['Train', 'Validation'], loc='upper right')

# Plot training & validation loss values
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['previous_illuminance_output_loss'])
plt.plot(history.history['val_previous_illuminance_output_loss'])
plt.title('Illuminance Task Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')

# Plot training & validation loss values
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['previous_illuminance_output_mse'])
plt.plot(history.history['val_previous_illuminance_output_mse'])
plt.title('Illuminance Task MSE')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.legend(['Train', 'Validation'], loc='upper right')

# Plot training & validation loss values
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['previous_illuminance_output_mae'])
plt.plot(history.history['val_previous_illuminance_output_mae'])
plt.title('Illuminance Task MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend(['Train', 'Validation'], loc='upper right')

# Plot training & validation loss values
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['image_retinex_output_loss'])
plt.plot(history.history['val_image_retinex_output_loss'])
plt.title('Retinex Task Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')

# Plot training & validation loss values
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['image_retinex_output_mse'])
plt.plot(history.history['val_image_retinex_output_mse'])
plt.title('Retinex Task MSE')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.legend(['Train', 'Validation'], loc='upper right')

# Plot training & validation loss values
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['image_retinex_output_mae'])
plt.plot(history.history['val_image_retinex_output_mae'])
plt.title('Retinex Task MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend(['Train', 'Validation'], loc='upper right')

plt.tight_layout()
plt.show()